# Project 3 - Part 1

## Imports and Load data

In [40]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load Data
basics = pd.read_csv('Data/title.basics.tsv.gz', sep='\t', low_memory=False)
ratings = pd.read_csv('Data/title.ratings.tsv.gz', sep='\t', low_memory=False)
akas = pd.read_csv('Data/title-akas-us-only.csv', low_memory=False)

## Clean the dataset

In [3]:
# Clean akas dataset
# File only contains US movies

# Replace "\N" with np.nan
akas.replace({'\\N':np.nan}, inplace=True)

In [4]:
# Fix Basics dataset

#Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)
keepers = basics['tconst'].isin(akas['titleId'])
keepers

basics = basics[keepers]


In [6]:
#Replace "\N" with np.nan
basics.replace({'\\N':np.nan}, inplace=True)
basics.head(20)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,NaN,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,NaN,1,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,NaN,1,"Documentary,Short"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,NaN,1,"Documentary,Short"
11,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,0,1896,NaN,1,"Documentary,Short"
12,tt0000013,short,The Photographical Congress Arrives in Lyon,Le débarquement du congrès de photographie à Lyon,0,1895,NaN,1,"Documentary,Short"


In [22]:
#Eliminate movies that are null for runtimeMinutes and genres
basics.dropna(subset=['runtimeMinutes','genres'], inplace=True)

In [23]:
#keep only titleType==Movie
is_movie = basics['titleType'].str.contains('movie',case=False)
basics = basics[is_movie]


In [41]:
#Convert the startYear column to float data type.
basics['startYear'] = basics['startYear'].astype(float)
basics.dtypes

tconst             object
titleType          object
primaryTitle       object
originalTitle      object
isAdult            object
startYear         float64
endYear            object
runtimeMinutes     object
genres             object
dtype: object

In [37]:
#Filter the dataframe using startYear. Keep years between 2000-2021 (Including 2000 and 2021)
basics = basics[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2021)]

In [38]:
#Eliminate movies that include "Documentary" in the genre (see tip below).
is_doc = basics['genres'].str.contains('Documentary',case=False)
basics = basics[~is_doc]

In [39]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90804 entries, 34800 to 10098598
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          90804 non-null  object 
 1   titleType       90804 non-null  object 
 2   primaryTitle    90804 non-null  object 
 3   originalTitle   90804 non-null  object 
 4   isAdult         90804 non-null  object 
 5   startYear       90804 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  90804 non-null  object 
 8   genres          90804 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.9+ MB


In [43]:
# Clean ratings dataset
# Keep only movies that were included in your final title basics dataframe above.
rkeepers = ratings['tconst'].isin(akas['titleId'])
rkeepers

ratings = ratings[rkeepers]

In [44]:
# Replace "\N" with np.nan (if any
ratings.replace({'\\N':np.nan}, inplace=True)

## Final output - info()

In [48]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1452564 entries, 0 to 1452563
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1452564 non-null  object
 1   ordering         1452564 non-null  int64 
 2   title            1452564 non-null  object
 3   region           1452564 non-null  object
 4   language         4018 non-null     object
 5   types            981678 non-null   object
 6   attributes       47016 non-null    object
 7   isOriginalTitle  1451222 non-null  object
dtypes: int64(1), object(7)
memory usage: 88.7+ MB


In [46]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90804 entries, 34800 to 10098598
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          90804 non-null  object 
 1   titleType       90804 non-null  object 
 2   primaryTitle    90804 non-null  object 
 3   originalTitle   90804 non-null  object 
 4   isAdult         90804 non-null  object 
 5   startYear       90804 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  90804 non-null  object 
 8   genres          90804 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.9+ MB


In [47]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505073 entries, 0 to 1339768
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         505073 non-null  object 
 1   averageRating  505073 non-null  float64
 2   numVotes       505073 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.4+ MB


##  Save and verify updated files

In [49]:
#Save updated files
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [50]:
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [51]:
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [52]:
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1990
1,tt0000002,5.8,264
2,tt0000005,6.2,2647
3,tt0000006,5.0,182
4,tt0000007,5.4,829
